In [1]:
import pandas as pd
from mtl_data_to_text import get_justification_statement
from evaluate import load

c:\Users\Prahlad\Documents\Columbia\Coursework\Natural Language Generation and Summarization\just-verdict\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
data_df = pd.read_pickle('dev_test_verdict.pkl')

In [10]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,Unnamed: 0,claim_kg_size,main_text_kg_size,clean_claim_triples,clean_main_text_triples,claim_evidence_1,entailment_scores_1,verdict,verdict_actual
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",NaN,3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.56622183, -0.4859991], [0.430...",-1,0
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...",NaN,3,6,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","[[0.6284045, -0.64509016, -0.4347026], [0.6175...",-1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",NaN,3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.64209604, -0.69231135, -0.32926825], [0.68...",-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",NaN,3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.1503326, -0.75800896, 0.63468295], [0.660...",-1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",NaN,3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662...",-1,1


In [13]:
def get_justification(row):
    claim_evidences = row['claim_evidence_1']
    claims = list(claim_evidences.keys())
    entailment_scores = row['entailment_scores_1']
    index_values = [-1, 1, 0]

    justification1s = []
    justification2s = []

    for i in range(len(claims)):
        which_verdict = entailment_scores[i].argmax()
        evidence = claim_evidences[claims[i]][0]
        claim = claims[i]
  
        justification1, justification2 = get_justification_statement(claim, evidence, index_values[which_verdict])

        justification1s.append(justification1)
        justification2s.append(justification2)

    justification_base = ' '.join(justification1s)
    justification_mtl = ' '.join(justification2s)
    # print(justification_base)
    # print(justification_mtl)
    row['justification_base'] = justification_base
    row['justification_mtl'] = justification_mtl

    if row.name%100==0:
        print(row.name)
    
    return row


In [29]:
just_df = data_df.apply(get_justification, axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [30]:
just_df

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,claim_kg_size,main_text_kg_size,clean_claim_triples,clean_main_text_triples,claim_evidence_1,entailment_scores_1,verdict,verdict_actual,justification_base,justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.56622183, -0.4859991], [0.430...",-1,0,A baby died at an unnamed medical facility and...,A baby died at an unnamed medical facility bec...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,3,6,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","[[0.6284045, -0.64509016, -0.4347026], [0.6175...",-1,1,"Because Topeka is located in Shawnee County, i...",Topeka is not located in Shawnee County. Bat f...
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.64209604, -0.69231135, -0.32926825], [0.68...",-1,-1,Germany banned pork from school canteens becau...,Germany banned pork from school canteen becaus...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.1503326, -0.75800896, 0.63468295], [0.660...",-1,1,The Coronavirus is in Canada and is unrelated ...,Coronavirus is found in Canada and is unrelate...
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662...",-1,1,The Wayne National Forest is not an example of...,The Wayne National Forest is not an instance o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,3295,Christmas miracle for Florida dog whose heart ...,NaN,A Florida man was hoping for a Christmas mirac...,Associated Press,The heart of Gerald Ford’s 1-year-old French b...,https://www.gainesville.com/news/20200102/holi...,true,"Animals, Health, Florida, General News, Gaines...",[{'head': 'Christmas miracle for Florida dog w...,...,3,12,[(Christmas miracle for Florida dog whose hear...,"[(Neo, animal breed, French bulldog), (Gerald ...",{'(Christmas miracle for Florida dog whose hea...,"[[0.6461027, -0.5098372,

In [31]:
print(just_df.iloc[5]['claim'])
print(just_df.iloc[5]['explanation'])
print(just_df.iloc[5]['label'])
print(just_df.iloc[5]['justification_base'])
print(just_df.iloc[5]['justification_mtl'])

School closures will have little impact on COVID-19 control, review finds.
School closures do not tend to help contain the spread of infections during outbreaks of disease such as COVID-19, but will have a big impact on how societies restart after lockdown, scientists said on Monday.
true
SARS-COV2 coronavirus is unrelated to COVID-19 pandemic. The SARS-COV2 coronavirus is unrelated to COVID-19 pandemic and has no effect on control.
The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.


In [32]:
print(just_df.iloc[6]['claim'])
print(just_df.iloc[6]['explanation'])
print(just_df.iloc[6]['label'])
print(just_df.iloc[6]['justification_base'])
print(just_df.iloc[6]['justification_mtl'])

The state of Michigan threatened local beavers with a $10,000 per day fine for failing to remove their dams.
After some wrangling the agency ultimately dropped the issue, but not before Stephen Tvedten found an inventive way of quickly pointing out both how ludicrous and humorous the situation was. In a way dusty legal language never could, such a letter serves to drive home the silliness of Michigan DEQ’s intractable posturing. The beavers are likely still ignorant of how close they came to being fined $10,000 a day for dam living expenses.
true
The Department of Environmental Quality is the parent organization of Michigan, which is not an instance of the state. Because Pierson is located in Michigan, threatened local beavers are not located there. Because dam is used by dam beavers.
The Department of Environmental Quality is the parent organization of the Michigan DEQ. The threatened local beavers are not located in Pierson, Michigan. Because dam is used by dam beavers.


In [33]:
print(just_df.iloc[7]['claim'])
print(just_df.iloc[7]['explanation'])
print(just_df.iloc[7]['label'])
print(just_df.iloc[7]['justification_base'])
print(just_df.iloc[7]['justification_mtl'])

More infected in fresh wave of China bird flu - WHO.
Another 23 people in China have been infected with the H7N9 strain of bird flu in recent days, the World Health Organisation (WHO) said on Monday, adding to at least 24 new cases last week and confirming a fresh surge in the virus.
true
The H7N9 bird flu virus originates from China, where the WHO is the authority. The H7N9 bird flu virus originates from China and does not have the cause of WHO. The fresh wave of China bird flu is unrelated to H7N9 bird flu virus and originates from China where the WHO is located.
China bird flu is unrelated to H7N9 bird flu virus and is under the authority of WHO. The H7N9 bird flu virus originates from China and was not caused by WHO. The fresh wave of China bird flu is unrelated to H7N9 bird flu virus and originates from China where the WHO is located.


In [34]:
just_df.to_pickle('justifications.pkl')